In [1]:
from ast import literal_eval
from scipy.spatial import distance
from joblib import Parallel, delayed, parallel_backend
import multiprocessing

In [2]:
def bits(n):
    b = []
    while n:
        b = [n & 1] + b
        n >>= 1
    if len(b) < 16:
        for i in range(16 - len(b)):
            b = [0] + b
    return b or [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

In [3]:
def get_geno_bits(geno,n_sample,num_reg):
    geno_bits = []
    
    for ind in range(n_sample):
        geno_bits.append([])
        for reg in range(num_reg):
            seq = bits(geno[ind * num_reg + reg])
            for bp in range(len(seq)):
                geno_bits[ind].append(seq[bp])
    
    return geno_bits
    

In [4]:
def get_otus(geno_bits,n_sample,num_reg):
    av_list = []
    for i in range(n_sample - 1):
        av_list.append(1 + i)
    
    otu = []
    
    rep = 0
    while(1):
        tmp = [[item for item in geno_bits[rep]]]
        rm = []
        for i in range(len(av_list)):
            num_diff = distance.cityblock(geno_bits[rep],geno_bits[av_list[i]])
                
            if num_diff/(num_reg * 15) < 1.0:
                tmp.append([item for item in geno_bits[av_list[i]]])
                rm.append(av_list[i])
                
        otu.append([[item for item in sublist] for sublist in tmp])
        
        tmp_av_list = []
        for i in range(len(av_list)):
            check = 0
            for j in range(len(rm)):
                if av_list[i] == rm[j]:
                    check = 1
            if check == 0:
                tmp_av_list.append(av_list[i])
        av_list = [tmp_av_list[i] for i in range(len(tmp_av_list))]
        if len(av_list) == 0:
            break
        rep = min(av_list)
        av_list.remove(rep)
    
    rm_otu = []
    for i in range(len(otu)):
        if len(otu[i]) <= 1:
            rm_otu.append(otu[i])
    
    for i in range(len(rm_otu)):
        otu.remove(rm_otu[i])
    
    loc_S_list = []
    loc_th_list = []
    loc_n_list = []
    loc_D_list = []
    
    for cls in range(len(otu)):
        dist = []
        bp_sum = [0 for i in range(num_reg * 16)]
        for bp in range(num_reg * 16):
            bp_sum[bp] = bp_sum[bp] + otu[cls][len(otu[cls]) - 1][bp]
        for ind1 in range(len(otu[cls]) - 1):
            for indx in range(len(otu[cls]) - (ind1 + 1)):
                ind2 = ind1 + indx + 1
                dist.append(distance.cityblock(otu[cls][ind1],otu[cls][ind2]))
            for bp in range(len(otu[cls][ind1])):
                bp_sum[bp] = bp_sum[bp] + otu[cls][ind1][bp]
        
        n = len(otu[cls])
        
        k = sum(dist)/len(dist)
        
        S = 0
        for bp in range(num_reg * 16):
            if bp_sum[bp] > 0 and bp_sum[bp] < n:
                S = S + 1
        
        a1 = 0
        a2 = 0
        for i in range(n - 1):
            a1 = a1 + 1/(i+1)
            a2 = a2 + 1/((i+1)**2)
        
        b1 = (n + 1)/(3*(n - 1))
        b2 = (2*(n**2 + n + 3))/(9*n*(n - 1))
        c1 = b1 - 1/a1
        c2 = b2 - (n+2)/(a1*n) + a2/(a1**2)
        e1 = c1/a1
        e2 = c2/(a1**2 + a2)
        
        if (e1*S + e2*S*(S - 1)) > 0:
            d = (k - S/a1)/((e1*S + e2*S*(S - 1))**0.5)
        else:
            d = 0
        
        loc_S_list.append(S)
        loc_th_list.append(k)
        loc_D_list.append(d)
    
    loc_n_list = [len(otu[i]) for i in range(len(otu))]
    
    return [loc_n_list,loc_S_list,loc_th_list,loc_D_list]

In [5]:
geno=[]
with open('Geno.out','r') as data_file: 
    for line in data_file: 
       geno.append(literal_eval(line))
data_file.close()

In [6]:
# Check that genotypes are okay, such that max < 2^16 = 65536
max([max(geno[i]) for i in range(len(geno))])

40960

In [7]:
len(geno)

1000

In [8]:
num_reps = len(geno)
num_reg = 100
n_sample = 100

num_cores = 12

In [9]:
with parallel_backend("loky", inner_max_num_threads=1):
    geno_bits = Parallel(n_jobs=num_cores)(delayed(get_geno_bits)(geno[i],n_sample,num_reg) for i in range(len(geno)))

In [10]:
with parallel_backend("loky", inner_max_num_threads=1):
    otu_res = Parallel(n_jobs=num_cores)(delayed(get_otus)(geno_bits[i],n_sample,num_reg) for i in range(len(geno)))

In [11]:
n_list = [[item for item in otu_res[i][0]] for i in range(len(otu_res))]
S_list = [[item for item in otu_res[i][1]] for i in range(len(otu_res))]
pi_list = [[item for item in otu_res[i][2]] for i in range(len(otu_res))]
D_list = [[item for item in otu_res[i][3]] for i in range(len(otu_res))]

In [12]:
import matplotlib.pyplot as plt
import math as m

In [13]:
sum([pi_list[i][0] for i in range(len(otu_res))])/(len(otu_res))

0.9771941414141404

In [14]:
from statistics import mean
from random import choices

In [15]:
data = [pi_list[i][0] for i in range(len(otu_res))]

In [16]:
means = sorted(mean(choices(data, k=len(data))) for i in range(1000))

In [17]:
print(f'The sample mean of {mean(data):.1f} has a 95% confidence '
      f'interval from {means[24]:.5f} to {means[974]:.5f}')

The sample mean of 1.0 has a 95% confidence interval from 0.92967 to 1.02622
